# Complexity Hotspots

1. You must consider only the app folder from the Mastodon repository
(i.e., https://github.com/mastodon/mastodon).

In [12]:
import pydriller
# load the repository into PyDriller from URL 
repo = pydriller.Repository('https://github.com/mastodon/mastodon')

import pickle
# save repo as pickle file
with open('repo.pkl', 'wb') as f:
    pickle.dump(repo, f)

In [14]:
# open the pickle file
with open('repo.pkl', 'rb') as f:
    repo_mastodon = pickle.load(f)

2. Decide on the granularity of your analysis of software entities (e.g., source code
files); describe why you selected this specific granularity.

In [36]:
# git ls-files | awk -F . '{print $NF}' | sort | uniq -c | sort -n -r | awk '{print $2,$1}' | head -10
# rb 895
# js 437
# haml 214
# json 178
# png 37
# erb 33
# scss 32
# svg 24
# woff2 5
# woff 5

In [16]:
# get the list of commits
commits = list(repo_mastodon.traverse_commits())

# get the list of modified files in the commits
modified_files = [commit.modified_files for commit in commits]

# find unique files
unique_files = set([file for files in modified_files for file in files])